<a href="https://colab.research.google.com/github/Marsilea-viz/marsilea-workshop/blob/main/Marsilea-trackplot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

In [ ]:
!pip install marsilea

# Task 2: Visualize ATAC-seq data with track plot

The final visualization should look like:

<img src="https://raw.githubusercontent.com/Marsilea-viz/marsilea-workshop/main/asset/example-track.png" width="500px">

# Import essential pacakges and load data

In [ ]:
%config InlineBackend.figure_format = 'retina'
from sklearn.preprocessing import normalize
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt

import marsilea as ma
import marsilea.plotter as mp
import mpl_fontkit as fk
fk.install("Lato", verbose=False)

tracks = ma.load_data("track")

In [ ]:
tracks

Let's define some variables to use later

In [ ]:
colors = {
    "H3K9me1": "#DD6E42",
    "H3K9me2": "#E8DAB2",
    "H3K9me3": "#4F6D7A",
    "Background": "#C0D6DF",
}

lims = {
    "H3K9me1": 20,
    "H3K9me2": 35,
    "H3K9me3": 35,
    "Background": 20,
}

TRACK_HEIGHT = 0.5
TRACK_PAD = 0.1

Since we have 8 tracks, we can draw then by itering through the table.

We create a special empty canvas called `ZeroHeight`, it's invisible but you can add plots upon it. 

In [ ]:
canvas = ma.ZeroHeight(5, name="track")

for _, row in tracks.iterrows():
    cond, enz, track = row
    name = f"{cond}{enz}"
    color = colors[enz]
    canvas.add_bottom(
        mp.Area(track, color=color, add_outline=False, alpha=1, 
                label=cond, label_loc="right"),
        size=TRACK_HEIGHT,
        pad=TRACK_PAD,
        name=name,
    )
canvas.render()

To add experiment labels on the right, we can do a little trick by creating a new canvas and concatenate them later.

In [ ]:
enz_canvas = ma.ZeroHeight(1, name="enz")
for enz in tracks["enz"].drop_duplicates():
    enz_canvas.add_bottom(
        mp.Title(f"‎ ‎ {enz}", align="left"),
        size=TRACK_HEIGHT * 2,
        pad=TRACK_PAD * 2,
        name=enz,
    )

enz_canvas.render()

Now let's put the two canvas together, we can also add some space between plots to display the labels

In [ ]:
comp = canvas + 0.8 +  enz_canvas
comp.render()

However, we need to adjust the axis and add the blacklines, this is not available through marsilea. No worry! You can always retrive the `Axes` object to make customizations as you want. Just remember to assign a name to the canvas you want to retrieve.

In [ ]:
comp = canvas + 0.8 +  enz_canvas
comp.render()

# Add a line for enz
for enz in tracks["enz"].drop_duplicates():
    ax = comp.get_ax("enz", enz)
    ax.axvline(x=0, color="k", lw=4)

# Modify the limits
for _, row in tracks.iterrows():
    name = f"{row['cond']}{row['enz']}"
    lim = lims[row["enz"]]
    ax = comp.get_ax("track", name)
    ax.set_ylim(0, lim)
    ax.set_yticks([lim])